<right>
    <img src<a href="https://tukuz.com/national-college-of-ireland-nci-logo-vector-svg/" target="_blank"><img src="https://tukuz.com/wp-content/uploads/2020/05/national-college-of-ireland-nci-logo-vector.png" /></a> 
</right>

<span style="font-size:16px;"> 
<div style="text-align: center">
<br><b>National College of Ireland</b>
<br><b>Project Submission Python code</b>
<br>
<br>
<br><b>Student Name:</b>	<br>Barrios Cabrera Fabiola
<br>………………………………………………………………………………………………………………
<br>
<br><b>Student ID:</b>	<br>23344555
<br>………………………………………………………………………………………………………………
<br>
<br><b>Programme:</b>	<br>Master of Science in AI for Business
<br>………………………………………………………………
<br>
<br><b>Year:</b>	<br>2025
<br>………………………
<br>
<br><b>Module:</b>	<br>Intelligent Agents and Process Automation
<br>………………………………………………………………………………………………………………
<br>
<br><b>Lecturer:</b>	<br>Setlur Arun S
<br>………………………………………………………………………………………………………………
<br>
<br><b>Submission Due Date:</b>	<br>22/04/2025
<br>………………………………………………………………………………………………………………
<br>
<br><b>Project Title:</b>	<br>Automation of Onboarding Process in Human Resources for Technological Organisations
<br>………………………………………………………………………………………………………………



# Importing the libraries

In [1]:
#Importing the libraries
import csv
import os
import getpass
import smtplib
from email.mime.text import MIMEText

# 1. Detect CSV encoding

In [2]:
# 1. Detect CSV encoding

# Try to import chardet to guess the real file encoding
try:
    import chardet
    # Open the CSV in binary mode and read a sample of bytes
    with open('new_hire.csv', 'rb') as f:
        sample = f.read(10000)
    # chardet.detect returns a dict with the estimated encoding
    encoding = chardet.detect(sample)['encoding'] or 'utf-8'
    print(f"Detected encoding: {encoding}")
except ImportError:
    # If chardet is not installed, fall back to latin-1
    encoding = 'latin-1'
    print("chardet not found, defaulting to latin-1")

Detected encoding: UTF-8-SIG


# 2. Read the CSV and show available columns

In [3]:
# 2. Read the CSV and show available columns
# Open the CSV with the detected (or fallback) encoding
with open('new_hire.csv', newline='', encoding=encoding, errors='replace') as csvfile:
    reader = csv.DictReader(csvfile)
    # reader.fieldnames contains the list of column names
    print("CSV columns:", reader.fieldnames)
    try:
        # Take the first row as the new hire record
        new_hire = next(reader)
    except StopIteration:
        # If there are no rows, exit with an error
        raise SystemExit("The CSV file is empty")

CSV columns: ['FirstName', 'LastName', 'PersonalEmail', 'Position', 'Department']


# 3. Validate required fields

In [4]:
# 3. Validate required fields
required_fields = ["FirstName", "LastName", "PersonalEmail"]
for field in required_fields:
    if field not in new_hire:
        # If any required column is missing, raise a KeyError with details
        raise KeyError(f"Missing column '{field}' in CSV. Available columns: {reader.fieldnames}")

In [5]:
# Extract and clean values from the row
first_name     = new_hire["FirstName"].strip()
last_name      = new_hire["LastName"].strip()
personal_email = new_hire["PersonalEmail"].strip()
# Generate a username: first letter of first name + last name, all lowercase
username       = (first_name[0] + last_name).lower()

# 4. Simulate Active Directory operations

In [6]:
# 4. Simulate Active Directory operations

# Here we just print messages; in production, replace with
# real calls (e.g., ldap3 or PowerShell) to create the user.
print(f"Creating AD user: {username}")
print(f"Adding {username} to the 'Developers' group")
# → Replace with your actual AD integration code (e.g., ldap3, PowerShell calls, etc.)

Creating AD user: fbarrios
Adding fbarrios to the 'Developers' group


# 5. Compose the welcome email

In [7]:
# 5. Compose the welcome email

# Email subject, using f-string for personalization
subject = f"Welcome to the Company, {first_name}!"
# HTML body of the message
body = f"""<p>Hi {first_name},</p>
<p>Welcome to <strong>Our Tech Company</strong>. Your network username is <strong>{username}</strong> (you will be prompted to change your password on first login).</p>
<p>Please find attached your first-day guide. If you have any questions, reply to this email or reach out via our internal chatbot.</p>
<p>We’re excited to have you on the team!</p>
<p><em>- HR Team</em></p>"""

In [8]:
# Create the MIMEText object with the HTML body
msg = MIMEText(body, 'html')
msg['Subject'] = subject

# 6. Configure Gmail SMTP

In [9]:
# 6. Configure Gmail SMTP
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT   = 587

In [10]:
smtp = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
smtp.ehlo()        # Identify ourselves to the SMTP server
smtp.starttls()    # Upgrade the connection to secure TLS
smtp.ehlo()         # Re-identify after TLS handshake

(250,
 b'smtp.gmail.com at your service, [80.233.41.7]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

# 7. Load and sanitize credentials

In [11]:
# 7. Load and sanitize credentials
# Read environment variables (if set) and strip whitespace
GMAIL_USER = os.environ.get("GMAIL_USER", "........").strip()  #(example with Gmail SMTP - in real environment use corporate server)
GMAIL_PASS = os.environ.get("GMAIL_PASS", "........").strip()  # Sender account password (ideally use App Password or Vault)


In [12]:
# If not defined, prompt the user to enter them
if not GMAIL_USER:
    GMAIL_USER = input("Enter your Gmail address: ").strip()
if not GMAIL_PASS:
      # getpass hides the password input
    GMAIL_PASS = getpass.getpass("Enter your Gmail App Password: ").strip()

# 8. Authenticate and send

In [13]:
# 8. Authenticate and send
try:
    # Attempt to log in with provided credentials
    smtp.login(GMAIL_USER, GMAIL_PASS)
    print("✅ Gmail authentication successful")
    
    # Set the From and To headers on the message
    msg['From'] = GMAIL_USER
    msg['To']   = personal_email
    
     # Send the email
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    print("Welcome email sent to", personal_email)
    
# Handle authentication errors specifically
except smtplib.SMTPAuthenticationError:
    print("❌ Authentication failed: check your Gmail credentials or App Password.")

# Catch any other unexpected exceptions    
except Exception as e:
    print("❌ Failed to send email:", e)
# Always close the SMTP connection
finally:
    smtp.quit()

✅ Gmail authentication successful
Welcome email sent to fabiola.barrios.cab@gmail.com
